In [1]:
import sys
sys.path.append('../modules/')
import pandas as pd
import matplotlib.pyplot as plt
import pulp
from mymodule import *
from pulp_func import *

In [2]:
result1 = df21.groupby(['作物编号', '地块类型', '种植季次'])['种植面积/亩'].sum().reset_index()
result1.columns = ['作物编号', '地块类型', '种植季次', '种植面积/亩']
merged_df = pd.merge(result1, df22[['作物编号', '作物名称','作物类型' , '地块类型', '种植季次', '亩产量/斤', '种植成本/(元/亩)', '销售单价/(元/斤)']], on=['作物编号', '种植季次', '地块类型'], how='left')
merged_df['单地块类型总产量/斤'] = merged_df['种植面积/亩'] * merged_df['亩产量/斤']
merged_df['销售单价平均值'] = merged_df['销售单价/(元/斤)'].apply(calculate_average_price)
merged_df['利润'] = (merged_df['销售单价平均值'] * merged_df['单地块类型总产量/斤']) - (merged_df['种植成本/(元/亩)'] * merged_df['种植面积/亩'])
merged_df['每亩利润'] = merged_df['利润'] / merged_df['种植面积/亩']
# print(merged_df)
total_profit_2023 = merged_df.groupby(['作物编号', '作物名称'])[['利润', '每亩利润']].sum().reset_index()
total_profit_2023.columns = ['作物编号', '作物名称', '总利润/元', '每亩总利润/元']
# print(total_profit)
total_yield_2023 = merged_df.groupby(['作物编号', '作物名称'])['单地块类型总产量/斤'].sum().reset_index()
total_yield_2023.columns = ['作物编号', '作物名称', '总产量/斤']
# print(total_yield_2023)

In [3]:
fields = create_fields(df11)
# for field in fields:
#     print(field)
new_fields = create_new_fields(fields)
# for field in new_fields:
#     print(field)
crops = create_crops(df12, df22)
# for crop in crops:
#     print(crop)    

In [4]:
for year in range(2025, 2031):
    k = 0.5
    min_area_percent = 0.2
    max_plots = 4
    list = [k, min_area_percent, max_plots, year]
    expected_sales = total_yield_2023
    df_last1_result = pd.read_excel(f'../excels/附件3/{year - 1}_result_{k}.xlsx')
    df_last1_result.columns = [column.rstrip(' ') for column in df_last1_result.columns]
    if year - 2 == 2023:
        df_last2_result = pd.read_excel(f'../excels/附件3/2023_result.xlsx')
    else:    
        df_last2_result = pd.read_excel(f'../excels/附件3/{year - 2}_result_{k}.xlsx')
    df_last2_result.columns = [column.rstrip(' ') for column in df_last2_result.columns]
    optimize_planting_strategy(new_fields, crops, expected_sales, df_last1_result, df_last2_result, list)
    # 清空当年的种植信息
    for field in new_fields:
        field.planted_crop = []

Status: Optimal


e:\Users\Timothy\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


KeyboardInterrupt: 

In [5]:
for field in new_fields:
    print(field)

Field(planted_crop=[], field_name=A1, field_type=平旱地, field_area=80.0, crop_and_season=[['粮食', '单季'], ['粮食（豆类）', '单季']], season=单季, seasonmonth=[])
Field(planted_crop=[], field_name=A2, field_type=平旱地, field_area=55.0, crop_and_season=[['粮食', '单季'], ['粮食（豆类）', '单季']], season=单季, seasonmonth=[])
Field(planted_crop=[], field_name=A3, field_type=平旱地, field_area=35.0, crop_and_season=[['粮食', '单季'], ['粮食（豆类）', '单季']], season=单季, seasonmonth=[])
Field(planted_crop=[], field_name=A4, field_type=平旱地, field_area=72.0, crop_and_season=[['粮食', '单季'], ['粮食（豆类）', '单季']], season=单季, seasonmonth=[])
Field(planted_crop=[], field_name=A5, field_type=平旱地, field_area=68.0, crop_and_season=[['粮食', '单季'], ['粮食（豆类）', '单季']], season=单季, seasonmonth=[])
Field(planted_crop=[], field_name=A6, field_type=平旱地, field_area=55.0, crop_and_season=[['粮食', '单季'], ['粮食（豆类）', '单季']], season=单季, seasonmonth=[])
Field(planted_crop=[], field_name=B1, field_type=梯田, field_area=60.0, crop_and_season=[['粮食', '单季'], ['粮食（豆类）', 